In [6]:
import numpy as np
import xgboost as xgb
import csv

ModuleNotFoundError: No module named 'xgboost'

In [4]:
class KNearestNeighbor(object):
    """ a kNN classifier with L2 distance """

    def __init__(self):
        pass

    def train(self, X, y):
        """
        Train the classifier. For k-nearest neighbors this is just 
        memorizing the training data.

        Inputs:
        - X: A numpy array of shape (num_train, D) containing the training data
          consisting of num_train samples each of dimension D.
        - y: A numpy array of shape (N,) containing the training labels, where
             y[i] is the label for X[i].
        """
        self.X_train = X
        self.y_train = y

    def predict(self, X, k=1, num_loops=0):
        """
        Predict labels for test data using this classifier.

        Inputs:
        - X: A numpy array of shape (num_test, D) containing test data consisting
             of num_test samples each of dimension D.
        - k: The number of nearest neighbors that vote for the predicted labels.
        - num_loops: Determines which implementation to use to compute distances
          between training points and testing points.

        Returns:
        - y: A numpy array of shape (num_test,) containing predicted labels for the
          test data, where y[i] is the predicted label for the test point X[i].  
        """
        if num_loops == 0:
            dists = self.compute_distances_no_loops(X)
        elif num_loops == 1:
            dists = self.compute_distances_one_loop(X)
        elif num_loops == 2:
            dists = self.compute_distances_two_loops(X)
        else:
            raise ValueError('Invalid value %d for num_loops' % num_loops)

        return self.predict_labels(dists, k=k)

    def compute_distances(self, X):
        """
        Compute the distance between each test point in X and each training point
        in self.X_train using no explicit loops.

        Input / Output: Same as compute_distances_two_loops
        """
        num_test = X.shape[0]
        num_train = self.X_train.shape[0]
        dists = np.zeros((num_test, num_train)) 
        # expand the square differences into quadratic form, and re-arrange the terms
        # get dist_11 = sqrt(x1^2...+x1_tr^2...-2(x1*x1_tr+...)) across first row, sum over columns
        X_terms = np.sum(np.square(X), axis=1)
        Xtr_terms = np.sum(np.square(self.X_train), axis=1)
        X_Xtr = np.dot(X,np.transpose(self.X_train)) #transpose to element wise multiply x12 with x_tr12 e.g.

        dists = np.sqrt(X_terms[:,np.newaxis]+Xtr_terms[np.newaxis,:]-2*X_Xtr)
        return dists

    def predict_labels(self, dists, k=1):
        """
        Given a matrix of distances between test points and training points,
        predict a label for each test point.

        Inputs:
        - dists: A numpy array of shape (num_test, num_train) where dists[i, j]
          gives the distance betwen the ith test point and the jth training point.

        Returns:
        - y: A numpy array of shape (num_test,) containing predicted labels for the
          test data, where y[i] is the predicted label for the test point X[i].  
        """
        num_test = dists.shape[0]
        y_pred = np.zeros(num_test)
        for i in range(num_test):
            # A list of length k storing the labels of the k nearest neighbors to
            # the ith test point.
            closest_y = np.array(self.y_train[np.argsort(dists[i,:],axis=0)[:k]])
            y_pred[i] = np.bincount(closest_y).argmax()

        return y_pred

In [5]:
# clean: personid, hhid, cohort, qn5a, qn13, ui_emprate, ui_lfp removed
data_pathname = "/Users/Jason/Documents/CS 221/project/dataset/ASR2016-clean.csv"
data_file = open(data_pathname, 'r')
reader = csv.reader(data_file)
headers = next(reader, None)

X = []
y = []

num_training = 3820
for row in reader:
    row = [float(item) if item !=' ' else 0.0 for item in row]
    y.append(row[-1] == float('2'))
    X.append(row[1:-1])
    
X_train = np.asarray(X[:num_training])
y_train = np.asarray(y[:num_training])
X_test = np.asarray(X[num_training + 1:])
y_test = np.asarray(y[num_training + 1:])

num_test = X_test.shape[0]

classifier = KNearestNeighbor()
classifier.train(X_train, y_train)

dists = classifier.compute_distances(X_test)
y_test_pred = classifier.predict_labels(dists, k=25)
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / num_test
print('Got %d / %d correct => accuracy: %f' % (num_correct, num_test, accuracy))
# Got 672 / 954 correct => accuracy: 0.704403

Got 672 / 954 correct => accuracy: 0.704403
